In [20]:
import pandas as pd
from ConectionSQL import SQL
from Tempo import DataHora

In [21]:
sql=SQL('Netfeira','sqlserver','MOINHO','192.168.0.252')

conectando=sql.ConexaoSQL()

In [22]:
data=DataHora()

data_atual=data.HoraAtual()

ano=data_atual.year

In [23]:
querys={
    
    'Vendedor':
    
    """
    
    SELECT * FROM netfeira.vw_vendedor
    
    """,
    
    'Supervisor':
    
    """
    
    SELECT * FROM netfeira.vw_supervisor
    
    """,
    
    'Carteira':
    
    """
    
    SELECT cd_clien AS 'ID Cliente',cd_vend AS 'ID Vendedor' FROM vend_cli
    
    """
    
    
}

In [24]:
tabelas_df=dict()

for tabela,query in querys.items():
    
    tabelas_df[tabela]=pd.read_sql(query,conectando)
    
    pass


In [25]:
carteira_df=tabelas_df['Carteira'].groupby(['ID Vendedor'],as_index=False).count()

carteira_df.rename(columns={'ID Cliente':'Contagem'},inplace=True)

carteira_df.columns

Index(['ID Vendedor', 'Contagem'], dtype='object')

In [26]:
tabelas_df['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro'],
      dtype='object')

In [27]:
tabelas_df['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente'],
      dtype='object')

In [28]:
tabelas_df['Vendedor']=tabelas_df['Vendedor'].merge(tabelas_df['Supervisor'],on='ID Equipe',how='inner')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria','Data de Cadastro', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente']]

In [29]:
tabelas_df['Vendedor']=tabelas_df['Vendedor'].merge(carteira_df,on='ID Vendedor',how='left')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria','Data de Cadastro', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente','Contagem']]

In [30]:
tabelas_df['Vendedor']['Gestor']=tabelas_df['Vendedor'].apply(lambda info: 'SIM' if info['ID Vendedor']==info['ID Sup'] else 'NÃO',axis=1)

tabelas_df['Vendedor']=tabelas_df['Vendedor'].loc[(tabelas_df['Vendedor']['Gestor']=='NÃO')&(tabelas_df['Vendedor']['Contagem'].isnull())]

In [33]:
tabelas_df['Vendedor']=tabelas_df['Vendedor'].loc[tabelas_df['Vendedor']['Data de Cadastro'].dt.year<ano]

vendedores=tabelas_df['Vendedor']['ID Vendedor'].tolist()

In [36]:
for v in vendedores:
    
    query="""
    
    UPDATE vendedor
    SET ativo=0
    WHERE cd_vend='{0}'
        
    """.format(v)
    
    sql.Salvar(query,conectando)
      
    #break
    
    pass

DIRCEU  
ANDREL  
CAROLINA
RONALDPA
SAFIRA  
VBARBOSA
TABATA  
AMAURI  
ANDREALS
PATIOCOM
EDILANE 
JULIANAM
JONATHAN
LUIZCARL
MENU    
EDSONLOP
EURIPESF
MAXMOREI
REENAGAS
REFIRMIN
REJERON 
RELUCINE
REMARCOS
REMIQUEI
REMYRELA
RESOLANG


In [35]:
tabelas_df['Vendedor']

,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Data de Cadastro,ID Sup,Supervisor,Email Sup,ID Gerente,Gerente,Email Gerente,Contagem,Gestor
17,DIRCEU,DIRCEU ALEXANDRE RIBEIRO,DIRCEU RIBEIRO,EQUIPE 2 (REP),,CLT,2021-06-12,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NÃO
30,ANDREL,ANDRE LEITE,ANDRE LEITE,EQUIPE 4 (VAR),COMERCIAL@DEMARCHIBRASIL.COM,CLT,2020-09-02,DOUGLASM,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NÃO
31,CAROLINA,CAROLINA MARTINS,CAROLINA MARTINS,EQUIPE 4 (VAR),,PJ,2021-02-02,DOUGLASM,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NÃO
37,RONALDPA,RONALDO PARDINI,RONALDO PARDINI,EQUIPE 4 (VAR),RONALDOPARDINI@HOTMAIL.COM,PJ,2017-10-26,DOUGLASM,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NÃO
38,SAFIRA,SAFIRA POLERI MACHADO,SAFIRA MACHADO,EQUIPE 4 (VAR),SAFIRA.DEMARCHIBRASIL@GMAIL.COM,CLT,2020-10-20,DOUGLASM,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NÃO
39,VBARBOSA,VENDEDOR BARBOSA,VENDEDOR BARBOSA,EQUIPE 4 (VAR),,PJ,2017-11-01,DOUGLASM,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NÃO
56,TABATA,TABATA CRISTINA ORSI GENTIL,TABATA GENTIL,EQUIPE 6 (CLT),,CLT,2020-12-28,CARLOSH,CARLOS PASSOS,CARLOS.HENRIQUE@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NÃO
57,AMAURI,AMAURI PEDRO RAMOS BOER,AMAURI BOER,EQUIPE 11 (LS),,REPRESENTANTE,2021-02-18,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NÃO
58,ANDREALS,ANDREA APARECIDA GONCALVES ALCANTARA,ANDREA ALCANTARA,EQUIPE 11 (LS),,PJ,2021-09-24,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NÃO
62,PATIOCOM,PATIO DE COMPRAS,PATIO COMPRAS,EQUIPE 8 (ATEND),,AVULSO,2018-08-24,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NÃO
